# What has the highest population in Washington out of battery EVs or plug-in EVs?


In [ ]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import scipy.stats as st
from config import gkey
import gmaps
import requests
import time

In [2]:
# Load the Size History by County csv to a DataFrame
size_history_data_path = "../cleaned_data/Electric_Vehicle_Population_Size_History_By_County_Cleaned.csv"
size_history_full_data = pd.read_csv(size_history_data_path)

In [3]:
size_history_full_data.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,January 31 2017,Juneau,AK,Passenger,0,1,1,49,50,2.00
1,January 31 2017,Dale,AL,Passenger,0,1,1,71,72,1.39
2,January 31 2017,Russell,AL,Passenger,1,0,1,53,54,1.85
3,January 31 2017,Maricopa,AZ,Passenger,2,0,2,1774,1776,0.11
4,January 31 2017,Pima,AZ,Passenger,0,1,1,439,440,0.23


In [4]:
# Refine to Washington st recent date in history (October 31 2021)
size_history_data_WA = size_history_full_data.loc[size_history_full_data["State"]=="WA"]
size_history_data_WA

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
99,January 31 2017,Adams,WA,Passenger,2,1,3,12457,12460,0.02
100,January 31 2017,Adams,WA,Truck,0,0,0,5290,5290,0.00
101,January 31 2017,Asotin,WA,Passenger,4,4,8,12419,12427,0.06
102,January 31 2017,Asotin,WA,Truck,0,0,0,6648,6648,0.00
103,January 31 2017,Benton,WA,Passenger,129,133,262,129540,129802,0.20
...,...,...,...,...,...,...,...,...,...,...
11608,October 31 2021,Whatcom,WA,Truck,23,0,23,42447,42470,0.05
11609,October 31 2021,Whitman,WA,Passenger,69,57,126,21414,21540,0.58
11610,October 31 2021,Whitman,WA,Truck,2,0,2,8338,8340,0.02
11611,October 31 2021,Yakima,WA,Passenger,282,166,448,163726,164174,0.27


In [20]:
# Washington State: Group dataframe into Counties, then summarise columns of interest
WA_county_data_BEV_count = size_history_data_WA.groupby("County")["Battery Electric Vehicles (BEVs)"].sum()
WA_county_data_PHEV_count = size_history_data_WA.groupby("County")["Plug-In Hybrid Electric Vehicles (PHEVs)"].sum()
WA_county_data_EV_count = size_history_data_WA.groupby("County")["Electric Vehicle (EV) Total"].sum()

WA_county_data_NonEV_count = size_history_data_WA.groupby("County")["Non-Electric Vehicle Total"].sum()
WA_county_data_total_count = size_history_data_WA.groupby("County")["Total Vehicles"].sum()

WA_county_data_percent = (WA_county_data_EV_count / WA_county_data_total_count)*100
WA_county_data_percent = WA_county_data_percent.map("{:.2f}".format)

# Assemble columns into dataframe
WA_county_data = pd.DataFrame({
    "WA Total Battery Electric Vehicles (BEVs)": WA_county_data_BEV_count,
    "WA Total Plug-In Hybrid Electric Vehicles (PHEVs)": WA_county_data_PHEV_count,
    "WA Total Electric Vehicle (EV)": WA_county_data_EV_count,
    "WA Total Vehicles": WA_county_data_total_count,
    "WA Percent Electric Vehicles": WA_county_data_percent
    })


# Sort data by descending order

WA_county_data_sorted_count = WA_county_data.sort_values("WA Total Electric Vehicle (EV)", ascending=False)
WA_county_data_sorted_percent = WA_county_data.sort_values("WA Percent Electric Vehicles", ascending=False)


# Display data
WA_county_data_sorted_count.head(5)




,WA Total Battery Electric Vehicles (BEVs),WA Total Plug-In Hybrid Electric Vehicles (PHEVs),WA Total Electric Vehicle (EV),WA Total Vehicles,WA Percent Electric Vehicles
County,,,,,
King,1150072,398960,1549032,93317040,1.66
Snohomish,215546,91945,307491,38713101,0.79
Pierce,125594,87600,213194,40491938,0.53
Clark,100542,54082,154624,23326669,0.66
Kitsap,69387,34442,103829,13816001,0.75


In [19]:
WA_county_data_sorted_count.tail(5)

,WA Total Battery Electric Vehicles (BEVs),WA Total Plug-In Hybrid Electric Vehicles (PHEVs),WA Total Electric Vehicle (EV),WA Total Vehicles,WA Percent Electric Vehicles
County,,,,,
Pend Oreille,431,271,702,902785,0.08
Wahkiakum,261,240,501,278819,0.18
Ferry,279,135,414,478798,0.09
Columbia,196,97,293,235612,0.12
Garfield,24,58,82,135841,0.06
